In [ ]:
import sqlite3
import pandas as pd
import datetime

In [ ]:
conn = sqlite3.connect('/Users/Brienna/Library/Messages/chat.db')
c = conn.cursor()

In [ ]:
c.execute('select * from message')
c.description

In [ ]:
cmd1 = 'SELECT ROWID, text, is_from_me, \
        datetime(date + strftime(\'%s\',\'2001-01-01\'), \'unixepoch\') as date_utc \
        FROM message WHERE handle_id=47'
c.execute(cmd1)
df_msg = pd.DataFrame(c.fetchall(), columns=['id', 'text', 'is_from_me', 'time'])

In [ ]:
df_msg

In [ ]:
df_msg['time'] = [datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=-4) for t in df_msg['time']]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import json, re

In [ ]:
print(len(df_msg.index))

In [ ]:
stopwords = json.load(open('stopwords.json', 'r'))

In [ ]:
frequencies = {}
for message in df_msg['text']:
    if message != None:
        words = message.split(" ")
        for word in words:
            word = re.sub(r'[^\w\s]','', word).lower().strip()
            if word not in stopwords and word != '':
                if word in frequencies:
                    frequencies[word] += 1
                else:
                    frequencies[word] = 1

frequencies_sorted = sorted(frequencies.items(), key=lambda kv: kv[1])

In [ ]:
print(frequencies_sorted)

Generate word cloud.

In [ ]:
from wordcloud import WordCloud

In [ ]:
fullTexts = ""
for message in df_msg['text']:
    if message != None:
        fullTexts += message.lower()

def generate_wordcloud(text):
    wordcloud = WordCloud(font_path = '/Library/Fonts/Verdana.ttf',
                         relative_scaling = 1.0).generate(text)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

generate_wordcloud(fullTexts)

Most active day:

In [ ]:
from collections import Counter

def mostCommon(lst):
    data = Counter(lst)
    return max(lst, key=data.get)

days = []
for time in df_msg['time']:
    days.append(time.split(" ")[0])

most_common_day = mostCommon(days)
print(most_common_day)


Number of texts on that day:

In [ ]:
days.count(most_common_day)

In [ ]:
from_me = df_msg['is_from_me']==1
my_days = []
for time in df_msg[from_me]['time']:
    my_days.append(time.split(" ")[0])

my_days.count(most_common_day)

Analysis ideas:
- Find longest stretch of talking.